In [2]:
#!pip install -qU ipython
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
cd titanic

/kaggle/working/titanic


In [25]:
import numpy as np
import pandas as pd

path = "/kaggle/input/competitions/titanic/train.csv"
df = pd.read_csv(path)

df["Pclass"] = df["Pclass"] - 1
df["Age"] = df["Age"].fillna(value=df["Age"].mean())

df = pd.get_dummies(df, columns=["Embarked", "Cabin"], dummy_na=True, dtype=np.int64)
df["Sex"] = df["Sex"].map({"male": 0, "female": 1})

df.drop(["Name", "Ticket", "PassengerId"], axis=1, inplace=True)

In [ ]:
tgts = df["Survived"]
df.drop(["Survived"], axis=1, inplace=True)
inps = df.to_numpy()

In [ ]:
from sklearn.model_selection import train_test_split

SEED = 63166

train_inps, eval_inps, train_tgts, eval_tgts = train_test_split(inps,
                                                                tgts,
                                                                test_size=0.2,
                                                                random_state=SEED, 
                                                                stratify=tgts)

In [ ]:
from torch.utils.data import TensorDataset

TensorDataset(train_inputs)